In [32]:
from itertools import count
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

In [33]:
import scipy as sp
import scipy.sparse as sparse
import torch, torch.nn as nn
import pandas as pd
import numpy as np
def pop_code(sample, cen, wid, P,gamma,T,T_enc):
    P = cen.shape[0]
   # n_feature = sample.shape[1]
    x_sample = sp.tile(sample, (P, 1))#The numpy.tile() function constructs a new array by repeating array – 'arr', the number of times we want to repeat as per repetitions.
    x_cen = sp.tile(cen, (1, n_feature))

    grf_act = sp.exp(- (x_sample - x_cen)**2 / (2 * (wid**2)))  # activation of the receptive fields

    spike_times = ((1 - grf_act.flatten('F')) * T_enc) #rounding off

    # ignore boundary spikes (spikes at params[T_pop]). These spikes are only occurring because of rounding.
    actual_spikes = grf_act.flatten('F') != 0
    n_actual_spikes = sp.sum(actual_spikes)
    #print("sp.arange",(sp.arange(n_feature*P)[actual_spikes],
    #spike_times[actual_spikes]))
    #print("spike_times",spike_times)
    #print("sp.ones",(sp.ones(n_actual_spikes)))
    #print( "last",(n_feature*P, T),dtype=int)
    return sparse.csr_matrix((sp.ones(n_actual_spikes), (sp.arange(n_feature*P)[actual_spikes],
                                            spike_times[actual_spikes])), shape=(n_feature*P, T), dtype=int)

In [34]:
def hid_out(spike_pattern,w,hid_thres,norm_psp):
  n_output=w.shape[1]
  (in_spike, in_feature) = spike_pattern.T.nonzero()
  #delay_by_tau = np.arange(T, dtype=float) / 300
  #norm_psp = np.triu(lg.toeplitz(delay_by_tau * np.exp(1 - delay_by_tau)))
  T=norm_psp.shape[1]
  #print('hi1')
  out_spike_pattern = sparse.lil_matrix((n_output, T))
  last_out_spike = -1 * sp.ones((1, n_output))
  while True:
      psp = sp.dot((w[in_feature, :] * (in_spike[:, sp.newaxis] > last_out_spike)).T, norm_psp[in_spike, :])
      #print(psp.shape)
      #print(psp>hid_thres[:,sp.newaxis])
      out_spike = sp.argmax(psp > hid_thres[:, sp.newaxis], axis=1)
      #print(out_spike)
      act_spike = psp[sp.arange(n_output), out_spike] > hid_thres
      #print(act_spike)
      if sp.sum(act_spike) == 0:
          break

      out_spike_pattern[sp.arange(n_output)[act_spike], out_spike[act_spike]] = 1
      last_out_spike[0, act_spike] = out_spike[act_spike]
      #print(out_spike_pattern.tocsr())

  return out_spike_pattern.tocsr()

In [35]:
import numpy as np

def spike_response(s, tau):
    x = s/tau
    LIF_norm = x * np.exp(1-x)
    LIF_norm[LIF_norm<0]=0

    return LIF_norm

In [36]:
def sefron_out(spike,w,thres,norm_psp,n_neuron,n_class,tau):

  W_sample = weight_syn[np.tile(np.arange(n_neuron), (1, n_class)),
                             np.array(np.tile(spiked_hid_neurons, (1, n_class)), dtype=np.int32) - 1,
                             np.ravel(np.tile(np.arange(n_class), (n_neuron, 1)), order='F')]
  wh = np.reshape(W_sample, (n_neuron, n_class), order='F')
  t = np.matmul(np.ones((n_neuron, 1)), np.arange(T+1).reshape(1, int(T+1)))


  V = np.matmul(wh.transpose(), spike_response(t - spike[:, np.newaxis], tau))
  firing = (V>thres[:, np.newaxis])
  firing_time = np.argmax(firing.transpose(), axis=0)
  firing_time[firing_time==0] = T - 1

  tc = firing_time

      #print(out_spike_pattern.tocsr())

  return tc,V

In [37]:
import pandas as pd
train=pd.read_csv('/content/ae_train_1.csv',header=None) #fix header issue
train_sh= train.sample(frac=1)
test=pd.read_csv('/content/ae_test_1.csv',header=None)
test_sh= test.sample(frac=1)
#test.sample(frac=1) is done to ensure the first row is not considered as header

In [38]:
import numpy as np
from copy import deepcopy
import warnings
warnings.filterwarnings("ignore")

def STDP_norm(r_time, tau):
    stdp = np.exp(-np.abs(r_time)/tau)

    g = deepcopy(r_time)
    g[g>0] = 1
    g[g<0] = 0
    g = g*stdp

    if (isinstance(np.sum(g, axis=0), np.ndarray)):
        pos_weight = g/((np.sum(g, axis=0))[np.newaxis, :])
    else:
        pos_weight = g/np.sum(g)

    temp = np.isnan(pos_weight)
    pos_weight[temp]=0

    g= deepcopy(r_time)
    g[g>0] = 0
    g[g<0] = 1
    g = g * stdp

    if (isinstance(np.sum(g, axis=0), np.ndarray)):
        neg_weight = -1 * (g/((np.sum(g, axis=0))[np.newaxis, :]))
    else:
        neg_weight = -1 * (g/np.sum(g))

    temp = np.isnan(neg_weight)
    neg_weight[temp] = 0

    Ut = pos_weight + neg_weight

    return Ut

In [39]:
def gaussian_function( A, B):
  result = []
  sigma=.55/0.01
  for i in range(A.shape[0]):
    result.append(np.exp((-np.power(A[i]-B, 2)) / (2*(sigma**2))))
  return np.array(result)

In [54]:
def minimum(a, b):

    if a <= b:
        return a
    else:
        return b

In [58]:
import scipy.linalg as lg
import sys
P=6
gamma=.7
T=600
T_b=300
T_enc = 300
cen = (2 * (np.arange(1,P+1, dtype=float) )- 3) / (2 * (P - 2))
tau=T/2
marg=-(sys.maxsize)- 1
wid = 1 / (gamma * (P - 2))
n_sample, n_feature =train_sh.shape
n_feature -= 1
n_class = np.size(np.unique(train_sh.iloc[:, -1]))
class_of_neuron=np.zeros(1500)
n_neuron=0
n_neurons_in_class=np.zeros(n_class)
hidden_list=[]
U_TID_list=[]
psp_2_list=[]
l_rate=0.5
theta_sefron=np.zeros(n_class)
delay_by_tau = np.arange(T, dtype=float) / 300
norm_psp = np.triu(lg.toeplitz(delay_by_tau * np.exp(1 - delay_by_tau)))
V_i=np.zeros((n_sample,n_feature*P),dtype=int)
hid_thres=np.zeros(1500)
T_input=T/3
alpha=.60
V_k=np.empty((n_sample,1500))
Aplus=1
weight_ik=np.empty((n_feature*P,1500))
out_size=0

no_epochs=1 #no of epochs
test_acc_hid=[]
train_acc_hid=[]
seen=[]
Tm_sef=0.05/.01
for e in range(no_epochs):
  count_acc=0
  CC_n_t=[]
  MC_n_t=[]
  count_tacc=0
  t_cc=-1
  t_mc=-1
  t_train=np.arange(T+1)
  #change the order of the input sample
  #cnf_mat = np.zeros((3, 3))

  for s in range(n_sample):
    CC_n=[]
    MC_n=[]

    curr_class=train_sh.iloc[s,-1]

    sample=train_sh.iloc[s,:-1]
    sample = sample[sp.newaxis, :]


    spike_pattern = pop_code(sample, cen[:, sp.newaxis], wid, P,gamma,T,T_enc)


    if ((n_neurons_in_class[curr_class-1]==0)and (e==0)):

      n_neurons_in_class[curr_class-1]+=1
      (neuron,spike_time)=spike_pattern.nonzero()
      class_of_neuron[n_neuron]=curr_class
      psp4spike = norm_psp[spike_time, 200]

      total_pre_psp = np.zeros(n_feature*P)
      np.add.at(total_pre_psp, neuron, psp4spike)
      weight_ik[:, n_neuron] = total_pre_psp / np.sum(total_pre_psp)
      hid_thres[n_neuron] = np.dot(weight_ik[:, n_neuron].T, total_pre_psp)

      n_neuron += 1
      weight_syn=np.zeros((int(n_neuron),int(T+1),int(n_class)))
    else:
      #computing hidden neuron output.

      #weight_syn.shape=(int(n_neuron),int(T+1),int(n_class))
      hidden=hid_out(spike_pattern,weight_ik[:,:n_neuron],hid_thres[:n_neuron],norm_psp)

      ## We know what time the neurons have spiked using the hidden variable above



      #index of hidden neurons that have spiked
      spiked_hid_neurons=np.argmax(hidden.toarray(),axis=1)
      hidden_list.append(spiked_hid_neurons)

      U_TID=STDP_norm((300-spiked_hid_neurons),tau)
      psp_2=norm_psp[spiked_hid_neurons,300]
      U_TID_list.append(U_TID)
      psp_2_list.append(psp_2)
      #print(spiked_hid_neurons)
      #print(spiked_hid_neurons.shape,U_TID.shape,psp_2.shape)
      if(theta_sefron[curr_class-1]==0):
        weight_syn[:,:,curr_class-1]=np.multiply((gaussian_function(t_train, spiked_hid_neurons.transpose())).transpose(), U_TID[:, np.newaxis])

        theta_sefron[curr_class-1] = (np.matmul(U_TID[np.newaxis, :], psp_2[:, np.newaxis])).squeeze()
      for i in range(n_neuron):
        if(class_of_neuron[i]==curr_class):

          CC_n.append(i)

      min=T+1
      CC=-1
      for i in CC_n:
        if(spiked_hid_neurons[i]<min):
          min=spiked_hid_neurons[i]
          CC=i
      t_cc=min
      CC=i


      for i in range(n_neuron):
        if(class_of_neuron[i]!=curr_class):

          MC_n.append(i)
      min_2=T+1
      MC=-1

      for i in MC_n:
        if(i<n_neuron):
          if(spiked_hid_neurons[i]<min_2):
            min_2=spiked_hid_neurons[i]
            MC=i
      t_mc=min_2
      MC=i

      #margin calculation

      if((t_mc!=-1) and(t_cc!=-1)):
        marg=t_mc-t_cc
      T_m=28
      T_a=alpha*T
      eta=0.001
      if (e==0):
        if((CC==-1) or ((CC!=-1)and (t_cc>T_a))and (n_neuron < 1500)):
          n_neurons_in_class[curr_class-1]+=1
          class_of_neuron[n_neuron]=curr_class
          spike_pattern = pop_code(sample, cen[:, sp.newaxis], wid, P,gamma,T,T_enc)
          (neuron,spike_time)=spike_pattern.nonzero()

          psp4spike = norm_psp[spike_time, 200]
          total_pre_psp = np.zeros(n_feature * P)
          np.add.at(total_pre_psp, neuron, psp4spike)

                  # initialize the weights and threshold
          weight_ik[:, n_neuron] = total_pre_psp / np.sum(total_pre_psp)
          hid_thres[n_neuron] = np.dot(weight_ik[:, n_neuron].T, total_pre_psp)
          n_neuron+=1


        elif((MC!=-1)and(marg<T_m)):
          (neuron, spike_time) = spike_pattern.nonzero()


      dummy,V=sefron_out(spiked_hid_neurons,weight_syn,theta_sefron,norm_psp,n_neuron,n_class,tau)

      Other_class = np.array(list(set(np.arange(n_class)) - set([curr_class-1])))
      tcc_sef=dummy[curr_class-1]
      tmc_sef=np.amin(dummy[Other_class])
      reference_time = deepcopy(dummy)

      if(tmc_sef<tcc_sef+Tm_sef):
        #300 is Td
        if (tcc_sef > 300-1):
                    reference_time[curr_class-1] = 300 - 1
        trf_mc = minimum(T-1, tcc_sef+Tm_sef)
        Wrng_class = np.where(dummy[Other_class] < tcc_sef+Tm_sef)[0]
        reference_time[Other_class[Wrng_class]] = trf_mc
        r_time = (dummy[np.newaxis, :]+1) - spiked_hid_neurons[:, np.newaxis]

        Ut = STDP_norm(r_time, tau)

        r_time = (reference_time[np.newaxis, :]+1) - spiked_hid_neurons[:, np.newaxis]

        Ut_de = STDP_norm(r_time, tau)

        w_tf = theta_sefron / np.sum(Ut * spike_response((dummy[np.newaxis, :]+1) - spiked_hid_neurons[:, np.newaxis], tau), axis=0)

        w_td = theta_sefron/ np.sum(Ut_de * spike_response((reference_time[np.newaxis, :]+1) - spiked_hid_neurons[:, np.newaxis], tau), axis=0)

        delta_W = np.multiply(Ut_de, (w_td - w_tf)[np.newaxis, :])

        delta_wx = np.multiply(gaussian_function(np.arange(T+1), np.tile(spiked_hid_neurons, (1, n_class)).squeeze()).transpose(), np.ravel(delta_W, order='F')[:, np.newaxis])
        weight_syn = weight_syn + l_rate * np.transpose(np.reshape(delta_wx.transpose(), (int(T+1), np.array(n_neuron, dtype=np.int32), n_class), order='F'), (1, 0, 2))
        weight_syn[weight_syn==-np.inf] = np.inf
        weight_syn[np.isnan(weight_syn)] = np.inf


      print(spiked_hid_neurons,dummy,curr_class,tcc_sef,tmc_sef,weight_syn.shape)





[319 227 274] [599 599 301 599] 3 301 599 (3, 601, 4)
[344 252 257] [599 599 305 599] 3 305 599 (3, 601, 4)
[203 316 380] [599 599 599 301] 4 301 599 (3, 601, 4)
[352 266 218] [301 599 320 599] 1 301 320 (3, 601, 4)
[348 259 269] [331 599 315 599] 3 315 331 (3, 601, 4)
[201 303 373] [535 599 599 299] 4 299 535 (3, 601, 4)
[357 267 208] [298 599 331 599] 1 298 331 (3, 601, 4)
[203 303 369] [536 599 599 301] 4 301 536 (3, 601, 4)
[368 287 211 339] [599 599 599 599] 1 599 599 (4, 601, 4)
[204 298 364 229] [599 599 599 599] 4 599 599 (4, 601, 4)
[370 292 201 342] [599 599 599 599] 1 599 599 (4, 601, 4)
[318 224 274 316] [599 599 599 599] 3 599 599 (4, 601, 4)
[371 292 210 342] [329 599 599 599] 1 329 599 (4, 601, 4)
[240 326 367 213] [599 301 599 599] 2 301 599 (4, 601, 4)
[202 321 381 232] [599 308 599 599] 4 599 308 (4, 601, 4)
[360 281 261 350] [599 599 599 599] 3 599 599 (4, 601, 4)
[217 337 391 260] [599 599 599 599] 4 599 599 (4, 601, 4)
[255 322 356 213] [599 316 599 599] 2 316 599 

In [14]:
n_neuron

4

In [28]:

for i in range(len(hidden_list)):
  while len(hidden_list[i])<n_neuron:
   hidden_list[i]=np.insert(hidden_list[i],len(hidden_list[i])-1,0)


In [29]:
hidden_list

[array([  0,   0,   0, 217]),
 array([303,   0,   0, 278]),
 array([205,   0,   0, 259]),
 array([280,   0,   0, 207]),
 array([241,   0,   0, 296]),
 array([271,   0,   0, 229]),
 array([277, 260,   0, 359]),
 array([211, 244, 289, 287]),
 array([312, 348, 218, 203]),
 array([208, 266, 298, 283]),
 array([289, 330, 226, 204]),
 array([204, 270, 327, 322]),
 array([315, 351, 234, 207]),
 array([336, 336, 212, 262]),
 array([224, 286, 345, 345]),
 array([212, 239, 290, 293]),
 array([269, 323, 363, 360]),
 array([321, 350, 216, 210]),
 array([251, 225, 340, 356]),
 array([314, 350, 224, 203]),
 array([347, 337, 232, 289]),
 array([255, 218, 333, 351]),
 array([335, 326, 219, 271]),
 array([291, 266, 288, 317]),
 array([339, 346, 214, 258]),
 array([225, 233, 303, 306]),
 array([270, 236, 352, 367]),
 array([303, 339, 224, 205]),
 array([268, 314, 237, 213]),
 array([267, 218, 344, 360]),
 array([202, 262, 321, 313]),
 array([303, 337, 231, 221]),
 array([325, 333, 202, 244]),
 array([33

In [ ]:
weight_syn

array([[[ 0.00000000e+00,  2.36163232e-04,  0.00000000e+00,
          5.29792976e-05],
        [ 0.00000000e+00,  2.52931390e-04,  0.00000000e+00,
          5.72685992e-05],
        [ 0.00000000e+00,  2.70800593e-04,  0.00000000e+00,
          6.18847093e-05],
        ...,
        [ 0.00000000e+00,  3.63501158e-12,  0.00000000e+00,
          2.06685209e-10],
        [ 0.00000000e+00,  3.19479009e-12,  0.00000000e+00,
          1.83343638e-10],
        [ 0.00000000e+00,  2.80695397e-12,  0.00000000e+00,
          1.62584344e-10]],

       [[ 0.00000000e+00, -2.56781240e-07,  0.00000000e+00,
         -3.44628590e-08],
        [ 0.00000000e+00, -2.83787158e-07,  0.00000000e+00,
         -3.82513829e-08],
        [ 0.00000000e+00, -3.13529651e-07,  0.00000000e+00,
         -4.24423488e-08],
        ...,
        [ 0.00000000e+00, -5.66211858e-07,  0.00000000e+00,
         -9.92814015e-07],
        [ 0.00000000e+00, -5.13516605e-07,  0.00000000e+00,
         -9.04294355e-07],
        [ 0.000

In [ ]:
theta_sefron

array([0.2385555 , 0.4063158 , 0.29746976, 0.56530122])

Below is Additional Old code please ignore

In [ ]:
# @title
import scipy.linalg as lg
import sys
P=6
gamma=0.7
P
T = 600
T_enc = 300
cen = (2 * (np.arange(1,P+1, dtype=float) )- 3) / (2 * (P - 2))
#print(cen)
tau=300
marg=-(sys.maxsize)- 1
wid = 1 / (gamma * (P - 2))
#print(wid)
n_sample, n_feature =train_sh.shape
n_feature -= 1
n_class = np.size(np.unique(train_sh.iloc[:, -1]))
class_of_neuron=np.zeros(1500)
n_neuron=0
n_neurons_in_class=np.zeros(n_class) #class size change here
weight_ik=np.empty((n_feature*P,1500))
#initializing values for SEFRON




V_i=np.zeros((n_sample,n_feature*P),dtype=int)
hid_thres=np.zeros(1500)
T_input=T/3
alpha=.60
V_k=np.empty((n_sample,1500))

delay_by_tau = np.arange(T, dtype=float) / 300
norm_psp = np.triu(lg.toeplitz(delay_by_tau * np.exp(1 - delay_by_tau)))


no_epochs=2 #no of epochs
test_acc_hid=[]
train_acc_hid=[]
for e in range(no_epochs):
  count_acc=0
  CC_n_t=[]
  MC_n_t=[]
  count_tacc=0
  t_cc=-1
  t_mc=-1
  #change the order of the input sample
  #cnf_mat = np.zeros((3, 3))
  for s in range(n_sample):
    CC_n=[]
    MC_n=[]

    curr_class=train_sh.iloc[s,-1]

    sample=train_sh.iloc[s,:-1]
    sample = sample[sp.newaxis, :]

    spike_pattern = pop_code(sample, cen[:, sp.newaxis], wid, P,gamma,T,T_enc)


    if ((n_neurons_in_class[curr_class-1]==0)and (e==0)):

      n_neurons_in_class[curr_class-1]+=1
      (neuron,spike_time)=spike_pattern.nonzero()
      class_of_neuron[n_neuron]=curr_class
      psp4spike = norm_psp[spike_time, 200]
      total_pre_psp = np.zeros(n_feature*P)
      np.add.at(total_pre_psp, neuron, psp4spike)
      weight_ik[:, n_neuron] = total_pre_psp / np.sum(total_pre_psp)
      hid_thres[n_neuron] = np.dot(weight_ik[:, n_neuron].T, total_pre_psp)
      n_neuron += 1

    else:
      #computing hidden neuron output.
      print(n_neuron)
      hidden=hid_out(spike_pattern,weight_ik[:,:n_neuron],hid_thres[:n_neuron],norm_psp)

      ## We know what time the neurons have spiked using the hidden variable above



      #index of hidden neurons that have spiked
      spiked_hid_neurons=np.argmax(hidden.toarray(),axis=1)


      fired_neurons = np.squeeze(np.asarray(hidden[np.arange(n_neuron), spiked_hid_neurons] == 1))

      #boolean of which neurons have fired
      spiked_hid_neurons[np.logical_not(fired_neurons)] = T+1

      for i in range(n_neuron):
        if(class_of_neuron[i]==curr_class):

          CC_n.append(i)

      min=T+1
      CC=-1
      for i in CC_n:
        if(spiked_hid_neurons[i]<min):
          min=spiked_hid_neurons[i]
          CC=i
      t_cc=min
      CC=i


      for i in range(n_neuron):
        if(class_of_neuron[i]!=curr_class):

          MC_n.append(i)
      min_2=T+1
      MC=-1

      for i in MC_n:
        if(i<n_neuron):
          if(spiked_hid_neurons[i]<min_2):
            min_2=spiked_hid_neurons[i]
            MC=i
      t_mc=min_2
      MC=i

      #margin calculation

      if((t_mc!=-1) and(t_cc!=-1)):
        marg=t_mc-t_cc
      T_m=28
      T_a=alpha*T
      eta=0.001
      if (e==0):
        if((CC==-1) or ((CC!=-1)and (t_cc>T_a))and (n_neuron < 1500)):
          n_neurons_in_class[curr_class-1]+=1
          class_of_neuron[n_neuron]=curr_class
          spike_pattern = pop_code(sample, cen[:, sp.newaxis], wid, P,gamma,T,T_enc)
          (neuron,spike_time)=spike_pattern.nonzero()

          psp4spike = norm_psp[spike_time, 200]
          total_pre_psp = np.zeros(n_feature * P)
          np.add.at(total_pre_psp, neuron, psp4spike)

                  # initialize the weights and threshold
          weight_ik[:, n_neuron] = total_pre_psp / np.sum(total_pre_psp)
          hid_thres[n_neuron] = np.dot(weight_ik[:, n_neuron].T, total_pre_psp)
          n_neuron+=1

        elif((MC!=-1)and(marg<T_m)):
          (neuron, spike_time) = spike_pattern.nonzero()

          if(t_cc<601):
            psp4spike = norm_psp[spike_time, t_cc]
            total_pre_psp = np.zeros(n_feature * P)
            np.add.at(total_pre_psp, neuron, psp4spike)
            weight_ik[:, CC] += eta * sp.maximum((total_pre_psp / np.sum(total_pre_psp)) - weight_ik[:, CC], 0)


          psp4spike = norm_psp[spike_time, t_mc]
          total_pre_psp = np.zeros(n_feature * P)
          np.add.at(total_pre_psp, neuron, psp4spike)
          weight_ik[:, MC] -= eta * sp.maximum((total_pre_psp / np.sum(total_pre_psp)) - weight_ik[:, MC], 0)

      elif(e!=0):
        if((MC!=-1)and(marg<T_m)):
          (neuron, spike_time) = spike_pattern.nonzero()

          if(t_cc<601):
            psp4spike = norm_psp[spike_time, t_cc]
            total_pre_psp = np.zeros(n_feature * P)
            np.add.at(total_pre_psp, neuron, psp4spike)
            weight_ik[:, CC] += eta * sp.maximum((total_pre_psp / np.sum(total_pre_psp)) - weight_ik[:, CC], 0)
            psp4spike = norm_psp[spike_time, t_mc]
            total_pre_psp = np.zeros(n_feature * P)
            np.add.at(total_pre_psp, neuron, psp4spike)
            weight_ik[:, MC] -= eta * sp.maximum((total_pre_psp / np.sum(total_pre_psp)) - weight_ik[:, MC], 0)




    if(t_cc<t_mc):
      count_acc+=1
     #change the s==79 value to the number of rows in training file minus 1
    if(s==65):
      acc=count_acc/(s+1)
      train_acc_hid.append(acc)


  test_sample,test_feat=test_sh.shape
  #count_acc=0
  for t in range(test_sample):
    curr_class_test=test_sh.iloc[t,-1]

    sample_test=test_sh.iloc[t,:-1]
    sample_test = sample_test[sp.newaxis, :]

    spike_pattern_test = pop_code(sample_test, cen[:, sp.newaxis], wid, P,gamma,T,T_enc)
    hidden_test=hid_out(spike_pattern_test,weight_ik[:,:n_neuron],hid_thres[:n_neuron],norm_psp)

      ## We know what time the neurons have spiked using the hidden variable above



      #index of hidden neurons that have spiked
    spiked_hid_neurons_test=np.argmax(hidden_test.toarray(),axis=1)


    fired_neurons_test = np.squeeze(np.asarray(hidden_test[np.arange(n_neuron), spiked_hid_neurons_test] == 1))

    #boolean of which neurons have fired
    spiked_hid_neurons_test[np.logical_not(fired_neurons_test)] = T+1

    for i in range(n_neuron):
      if(class_of_neuron[i]==curr_class_test):

        CC_n_t.append(i)

    min_t=T+1
    CC_t=-1
    for i in CC_n_t:
      if(spiked_hid_neurons_test[i]<min_t):
        min_t=spiked_hid_neurons_test[i]
        CC_t=i
    t_tcc=min_t
    CC_t=i


    for i in range(n_neuron):
      if(class_of_neuron[i]!=curr_class_test):

        MC_n_t.append(i)
    min_2_t=T+1
    MC_t=-1


    for i in MC_n_t:
      if(i<n_neuron):
        if(spiked_hid_neurons_test[i]<min_2_t):
          min_2_t=spiked_hid_neurons_test[i]
          MC_t=i
    t_tmc=min_2_t
    MC_t=i
    if(t_tcc<t_tmc):
      count_acc+=1
    #change the t==10 value to the number of rows in testing file minus 1
    if(t==64):
      acc=count_acc/(t+1)
      test_acc_hid.append(acc)

  '''for i in range(n_neuron):
    print(class_of_neuron[i])'''

1
True 1
4
[ True  True  True  True] 4
4
[ True  True  True  True] 4
4
[ True  True  True  True] 4
4
[ True  True  True  True] 4
4
[ True  True  True  True] 4
4
[ True  True  True  True] 4
4
[ True  True  True  True] 4
4
[ True  True  True  True] 4
4
[ True  True  True  True] 4
4
[ True  True  True  True] 4
4
[ True  True  True  True] 4
4
[ True  True  True  True] 4
4
[ True  True  True  True] 4
4
[ True  True  True  True] 4
4
[ True  True  True  True] 4
4
[ True  True  True  True] 4
4
[ True  True  True  True] 4
4
[ True  True  True  True] 4
4
[ True  True  True  True] 4
4
[ True  True  True  True] 4
4
[ True  True  True  True] 4
4
[ True  True  True  True] 4
4
[ True  True  True  True] 4
4
[ True  True  True  True] 4
4
[ True  True  True  True] 4
4
[ True  True  True  True] 4
4
[ True  True  True  True] 4
4
[ True  True  True  True] 4
4
[ True  True  True  True] 4
4
[ True  True  True  True] 4
4
[ True  True  True  True] 4
4
[ True  True  True  True] 4
4
[ True  True  True  True] 4
4